In [33]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.externals import joblib

from PIL import Image
import glob

import pandas as pd

import cv2

import mahotas

In [3]:
pd.set_option('display.max_columns', None)

In [59]:
for filename in glob.iglob('*.jpg'):
    im = Image.open(filename)
    imResize = im.resize((112,112), Image.ANTIALIAS)
    imResize.save(filename , 'JPEG', quality=90) 

In [9]:
image_list = []
for filename in glob.glob('*.png'):
    im = Image.open(filename)
    image_list.append(im)

In [10]:
image_list

[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1AF6B7F0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1AF6B710>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1AF6BBE0>]

In [30]:
image = image_list[0]

In [34]:
# chodzi o pih=gmetacja i połozenia pikseli
def fd_hu_moments(image):
    image = np.asarray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [35]:
# chodiz o teksturę, ile razy piksle przyjmuje jakąś wartośc obok innego pikslea o jakiejś wartości
def fd_haralick(image):
    image = np.asarray(image)
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

In [36]:
def fd_histogram(image, mask=None):
    image = np.asarray(image)
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [5,5,5],[0, 256, 0, 256, 0, 256])
    # normalize the histogram00
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [47]:
def pipline(image):
    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)
    global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
    return global_feature


In [39]:
aa = global_feature

In [40]:
type(aa)

numpy.ndarray

In [42]:
df = pd.DataFrame(data=aa.flatten())
df = df.transpose()

In [45]:
image_list = []
for filename in glob.glob('*.png'):
    im = Image.open(filename)
    image_list.append(im)
    
image_list

[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1E953A58>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1E9539B0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=112x112 at 0x9B1E953940>]

In [49]:
all_data = pd.DataFrame()                                   
for image in image_list:
    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)
    global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
    df = pd.DataFrame(data=global_feature.flatten())
    df = df.transpose()
    all_data = all_data.append(df, ignore_index = True)
    
all_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144
0,0.091848,0.042489,0.045543,0.042235,0.080908,0.081162,0.015266,0.011449,0.017810,0.000509,0.035620,0.001018,0.0,0.0,0.0,0.003816,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.013230,0.008905,0.008142,0.024425,0.082435,0.009668,0.006361,0.012467,0.021118,0.023662,0.004071,0.000254,0.000000,0.000254,0.000000,0.001272,0.000000,0.000000,0.000000,0.0,0.001272,0.0,0.0,0.0,0.0,0.042998,0.020354,0.024171,0.035111,0.038927,0.024934,0.053430,0.139426,0.390547,0.727918,0.010177,0.029005,0.154183,0.384186,0.268167,0.002035,0.007887,0.037401,0.006106,0.000254,0.001018,0.001781,0.000254,0.0,0.0,0.067932,0.006615,0.003816,0.003816,0.000254,0.021118,0.000000,0.000000,0.0,0.0,0.000509,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000261,460.717248,0.929708,3280.637965,0.157635,262.816601,12661.834610,8.616500,12.754662,0.000171,5.078042,-0.333288,0.996771,0.001139,2.593821e-08,7.091566e-13,9.340187e-13,-3.505212e-26,-5.672512e-17,-7.593504e-25
1,0.004674,0.107988,0.931276,0.016762,0.108472,0.000967,0.038199,0.017246,0.009671,0.083812,0.000000,0.000484,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000806,0.003707,0.004835,0.006286,0.002256,0.000322,0.002740,0.002579,0.045290,0.032074,0.000161,0.001451,0.008704,0.020147,0.002256,0.000161,0.000000,0.004352,0.000161,0.0,0.000161,0.0,0.0,0.0,0.0,0.001773,0.007575,0.006769,0.001934,0.001612,0.001289,0.008220,0.013378,0.044001,0.294469,0.000645,0.007414,0.030623,0.051899,0.059474,0.000484,0.007575,0.007253,0.000161,0.000000,0.000322,0.000645,0.000000,0.0,0.0,0.001612,0.008059,0.004835,0.000806,0.000161,0.000484,0.000161,0.000161,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004526,459.087846,0.882364,1949.222627,0.393827,272.921336,7337.802660,7.776773,10.596463,0.000512,4.151063,-0.454018,0.998871,0.001148,3.794797e-10,4.928374e-13,8.117070e-14,1.212294e-26,-1.459427e-18,-1.079851e-26
2,0.043192,0.017997,0.008798,0.005199,0.004799,0.187566,0.002400,0.000000,0.000000,0.000000,0.211162,0.000000,0.0,0.0,0.0,0.219560,0.0,0.0,0.0,0.0,0.337139,0.0,0.0,0.0,0.0,0.157971,0.233558,0.206363,0.008798,0.095583,0.335139,0.236757,0.154372,0.012398,0.045592,0.135176,0.110380,0.012798,0.001600,0.003199,0.041193,0.006399,0.000000,0.000000,0.0,0.018797,0.0,0.0,0.0,0.0,0.007599,0.015197,0.042792,0.047991,0.086384,0.015597,0.047991,0.137175,0.135575,0.293147,0.024396,0.070787,0.150773,0.326741,0.248355,0.016397,0.107980,0.192365,0.111980,0.008798,0.009998,0.037593,0.010398,0.0,0.0,0.000400,0.000400,0.001600,0.002400,0.006399,0.002799,0.000000,0.000000,0.0,0.0,0.000800,0.0,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000225,530.941675,0.926434,3607.296846,0.136380,164.163214,13898.245707,8.581939,12.817629,0.000152,5.211323,-0.316923,0.995818,0.001661,1.574723e-07,6.102989e-11,4.268747e-11,2.141980e-21,1.262561e-14,-3.989897e-22
